In [1]:
import pandas as pd
from pysam import FastaFile
import json
import logging
from crc64iso import crc64iso

In [2]:
logger = logging.getLogger(__name__)

In [3]:
protein_fa = FastaFile('/Users/liang/Box/MyCPTAC/CPTAC_proteome_v1/DCC/RefSeq_20180629/RefSeq.20180629_Human_ucsc_hg38_cpdbnr_mito_264contams.fasta.gz')

In [4]:
with open('../../custom_sources/cptac3_refseq_20180629/refseq_20180629_missing_uniparc_ids.list') as f:
    refseq_ids_without_uniparc = f.read().splitlines()
len(refseq_ids_without_uniparc)

239

In [5]:
prot_id = refseq_ids_without_uniparc[0]
prot_seq = protein_fa.fetch(prot_id)

In [6]:
df = pd.DataFrame({'original_protein_id': refseq_ids_without_uniparc})
df['uniparc_checksum'] = df['original_protein_id'].apply(protein_fa.fetch).apply(crc64iso.crc64)

In [7]:
df.to_csv('../../intermediates/refseq_20180629/ebi_api_missing/refseq_checksums.tsv', sep='\t', index=False)

```
tail -n+2 refseq_checksums.tsv \
    | parallel --bar -j5 --colsep \t \
        "curl -L 'https://www.uniprot.org/uniparc/?query=checksum%3A{2}&format=xml' > xmls/{1}.xml"
```